<h3 style = "font-size:60px; font-family:Garamond ; font-weight : normal; background-color: #f6f5f5 ; color : #fe346e; text-align: center; border-radius: 100px 100px;">Importing the libraries</h3>

In [1]:
import re
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

# For tokenization
import nltk
from nltk.tokenize.toktok import ToktokTokenizer
from nltk.corpus import stopwords
nltk.download('stopwords')

# For word lemmatization
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/niketan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/niketan/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /Users/niketan/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

<h3 style = "font-size:60px; font-family:Garamond ; font-weight : normal; background-color: #f6f5f5 ; color : #fe346e; text-align: center; border-radius: 100px 100px;">Importing the Dataset</h3>

In [2]:
originalData = pd.read_csv('data.tsv', sep="\t", on_bad_lines="warn")

b'Skipping line 83032: expected 6 fields, saw 7\n'
b'Skipping line 154657: expected 6 fields, saw 7\n'
b'Skipping line 323916: expected 6 fields, saw 7\n'


In [3]:
data = originalData

<h3 style = "font-size:60px; font-family:Garamond ; font-weight : normal; background-color: #f6f5f5 ; color : #fe346e; text-align: center; border-radius: 100px 100px;">Data Exploration</h3>

In [4]:
data.head()

,id,qid1,qid2,question1,question2,is_duplicate
0,133273,213221,213222.0,How is the life of a math student? Could you d...,Which level of prepration is enough for the ex...,0.0
1,402555,536040,536041.0,How do I control my horny emotions?,How do you control your horniness?,1.0
2,360472,364011,490273.0,What causes stool color to change to yellow?,What can cause stool to come out as little balls?,0.0
3,150662,155721,7256.0,What can one do after MBBS?,What do i do after my MBBS ?,1.0
4,183004,279958,279959.0,Where can I find a power outlet for my laptop ...,"Would a second airport in Sydney, Australia be...",0.0


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 363192 entries, 0 to 363191
Data columns (total 6 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   id            363192 non-null  object 
 1   qid1          363192 non-null  object 
 2   qid2          363185 non-null  float64
 3   question1     363181 non-null  object 
 4   question2     363180 non-null  object 
 5   is_duplicate  363180 non-null  float64
dtypes: float64(2), object(4)
memory usage: 16.6+ MB


<h3 style = "font-size:60px; font-family:Garamond ; font-weight : normal; background-color: #f6f5f5 ; color : #fe346e; text-align: center; border-radius: 100px 100px;">Data Preprocessing</h3>

<h4 style = "font-size:40px; font-family:Garamond ; font-weight : normal; background-color: #f6f5f5 ; color : #fe346e; text-align: center; border-radius: 100px 100px;">Dropping the rows with null values</h4>

In [6]:
data = data.dropna()

In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 363177 entries, 0 to 363191
Data columns (total 6 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   id            363177 non-null  object 
 1   qid1          363177 non-null  object 
 2   qid2          363177 non-null  float64
 3   question1     363177 non-null  object 
 4   question2     363177 non-null  object 
 5   is_duplicate  363177 non-null  float64
dtypes: float64(2), object(4)
memory usage: 19.4+ MB


<span style="color:brown; font-size:16px">The rows dropped from 363192 to 363177. There were 15 rows with null values</span>
<br> 

<h4 style = " font-family:Garamond ; font-weight : normal; background-color: #f6f5f5 ; color : #fe346e; border-radius: 100px 100px;"><span style="text-align:left; margin-left: 20px; padding:10px; font-size:30px;">Similarly dropping all the rows that has duplicate question2</span></h4>

In [8]:
data = data.drop_duplicates(["question2"])

In [9]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 272959 entries, 0 to 363190
Data columns (total 6 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   id            272959 non-null  object 
 1   qid1          272959 non-null  object 
 2   qid2          272959 non-null  float64
 3   question1     272959 non-null  object 
 4   question2     272959 non-null  object 
 5   is_duplicate  272959 non-null  float64
dtypes: float64(2), object(4)
memory usage: 14.6+ MB


<span style="color:brown; font-size:16px">Now the rows dropped from 363177 to 272959. There were 90218 rows where the question2 values were equal.</span>
<br> 

<h3 style = "font-size:60px; font-family:Garamond ; font-weight : normal; background-color: #f6f5f5 ; color : #fe346e; text-align: center; border-radius: 100px 100px;">Text Preprocessing</h3>
<h4 style="font-size:25px; font-family:Garamond ; font-weight : normal; background-color: #f6f5f5 ; color : #fe346e; border-radius: 30px 30px">
    <br>
    <ol>1. Data Cleaning</ol>
    <ol>2. Tokenization and stop words removal</ol>
    <ol>3. Lemmatization</ol>
</h4>

<h4 style = "font-size:30px; font-family:Garamond ; font-weight : normal; background-color: #f6f5f5 ; color : #fe346e; text-align: center; border-radius: 100px 100px;">1. Data Cleaning</h4>
<h5 style="font-size:25px; font-family:Garamond ; font-weight : normal; background-color: #f6f5f5 ; color : #fe346e; border-radius: 30px 30px">
    <br>
    <ol>1. Removing punctuation</ol>
    <ol>2. Removing numbers</ol>
    <ol>3. Removing html tags</ol>
    <ol>4. Removing urls</ol>
    <ol>5.Converting the text to lower case</ol>
</h5>

In [10]:
def remove_punctuation(q):
    return re.sub(r'[^\w\s]', "", q)

def remove_numbers(q):
    return re.sub(r'[\d+]', "", q)

def remove_html_tags(q):
    """
        This takes care of the html tags as well as &nsbm similar characters
        which are not specifically enclosed in html tags
    """  
    c_rule = re.compile('<.*?>|&([a-z0-9]+|#[0-9]{1,6}|#x[0-9a-f]{1,6});')
    return re.sub(c_rule,'', q)

def remove_urls(q):
    return re.sub('https://.*', '', q)


def convert_to_lowercase(q):
    return q.lower()

In [11]:
%%time
data["question2"] = data.question2.apply(remove_punctuation)
data["question2"] = data.question2.apply(remove_numbers)
data["question2"] = data.question2.apply(remove_html_tags)
data["question2"] = data.question2.apply(remove_urls)
data["question2"] = data.question2.apply(convert_to_lowercase)

CPU times: user 1.09 s, sys: 16.5 ms, total: 1.1 s
Wall time: 1.1 s


<h4 style = "font-size:30px; font-family:Garamond ; font-weight : normal; background-color: #f6f5f5 ; color : #fe346e; text-align: center; border-radius: 100px 100px;">2. Tokenization and Stop Words Removal</h4>

In [12]:
# getting the stopwords list
stopwords_list = set(stopwords.words("english"))
def apply_tokenization_and_remove_stopwords(review):
    # Applying tokenization
    tokenizer = ToktokTokenizer()
    tokens = tokenizer.tokenize(review)
    tokens = [token.strip() for token in tokens]
    # applying removal of stopwords
    review_no_stopwords = [word for word in tokens if word not in stopwords_list]
    return " ".join(review_no_stopwords)

In [13]:
data["question2"] = data.question2.apply(apply_tokenization_and_remove_stopwords)

<h4 style = "font-size:30px; font-family:Garamond ; font-weight : normal; background-color: #f6f5f5 ; color : #fe346e; text-align: center; border-radius: 100px 100px;">3. Lemmatization</h4>

In [14]:
# Converting the text to root word - eg. low, lower, lowest converted to low
def apply_lemmatization(review):
    lemmatized_review = []
    lemmatizer = WordNetLemmatizer()
    tokenizer = ToktokTokenizer()
    tokens = tokenizer.tokenize(review)
    tokens = [token.strip() for token in tokens]
    for w in tokens:
        lemmatized_review.append(lemmatizer.lemmatize(w))
    return " ".join(lemmatized_review)

In [15]:
data["question2"] = data.question2.apply(apply_lemmatization)

<h3 style = "font-size:60px; font-family:Garamond ; font-weight : normal; background-color: #f6f5f5 ; color : #fe346e; text-align: center; border-radius: 100px 100px;">Inverted File Mapping</h3>
<h5 style="font-size:25px; font-family:Garamond ; font-weight : normal; background-color: #f6f5f5 ; color : #fe346e; border-radius: 30px 30px">
    <br>
    <ol>1. First Calculate TF IDF Score</ol>
    <ol>2. Map the word with (doc, score) in a inverted file map</ol>
</h5>

In [16]:
class MapInvertFile:
    def __init__(self):
        self.N = 0
        self.invertFile = {} # word - (docid, tfidf score)
        self.numberOfWordInDoc = {}
        self.wordInDocuments = {}
        self.tfWord = {}
    
    def getCountTermInDocument(self, ids, questions):
        for doc, q in zip(ids, questions):
            for w in q.split():
                if (doc, w) not in self.numberOfWordInDoc:
                    self.numberOfWordInDoc[(doc, w)] = 1
                else:
                    self.numberOfWordInDoc[(doc,w)] += 1
                    
                # mapping document in which word exists
                if w not in self.wordInDocuments:
                    self.wordInDocuments[w] = {doc}
                else:
                    self.wordInDocuments[w].add(doc)
    
    # This function only calculate tf
    def calculateTF(self):
        for (doc, w) in self.numberOfWordInDoc:
            self.tfWord[(doc, w)] = np.log(1 + self.numberOfWordInDoc[(doc, w)])
    
    def setInvertedFile(self):
        for (doc, w) in self.tfWord:
            # calculate idf score
            idf = self.N/len(self.wordInDocuments[w])
            
            # calculate tfidf score
            self.tfWord[(doc, w)] = self.tfWord[(doc, w)] * idf
            tfIdfScore = self.tfWord[(doc, w)]
            
            # converting to invert file data structure
            if w not in self.invertFile:
                self.invertFile[w] = [(doc, tfIdfScore)]
            else:
                self.invertFile[w].append((doc, tfIdfScore))
            
    def getInvertedFile(self, data):
        ids = data.id.to_list()
        questions = data.question2.to_list()
        self.N = len(ids)
        self.getCountTermInDocument(ids, questions)
        self.calculateTF()
        self.setInvertedFile()
        return self.invertFile

<h3 style = "font-size:30px; font-family:Garamond ; font-weight : normal; background-color: #f6f5f5 ; color : #fe346e; text-align: center; border-radius: 100px 100px;"> Get the Inverted File Mapping</h3>

In [17]:
%%time
instance = MapInvertFile()
invertedFileMap = instance.getInvertedFile(data)

CPU times: user 3.12 s, sys: 95.9 ms, total: 3.21 s
Wall time: 3.21 s


<h3 style = "font-size:60px; font-family:Garamond ; font-weight : normal; background-color: #f6f5f5 ; color : #fe346e; text-align: center; border-radius: 100px 100px;">Text Matching for Inverted File</h3>
<br>
<br>
<span style="font-size:25px; font-family:Garamond ; font-weight : normal; background-color: #f6f5f5 ; color : #fe346e; border-radius: 30px 30px">For this we have to match first 100 queries in question1 where is_duplicate = True</span>

In [18]:
dataH = data[data.is_duplicate == 1.0][:100]

In [19]:
dataH

,id,qid1,qid2,question1,question2,is_duplicate
1,402555,536040,536041.0,How do I control my horny emotions?,control horniness,1.0
3,150662,155721,7256.0,What can one do after MBBS?,mbbs,1.0
7,106969,147570,787.0,What is the best self help book you have read?...,top self help book read,1.0
11,233239,71243,177376.0,What will be Hillary Clinton's policy towards ...,hilary clinton policy towards india become pre...,1.0
13,11568,22332,22333.0,Which is the best book to study TENSOR for gen...,best book tensor calculus,1.0
...,...,...,...,...,...,...
251,211943,108779,150467.0,Is there a directory of landline individual ph...,directory individual landline cell phone telep...,1.0
259,351813,480691,480692.0,Which operating system do Google engineers use?,operating system programmer developer google use,1.0
262,212168,317182,317183.0,What is 0 divided by infinity?,divided,1.0
268,402628,85510,23367.0,Is there proof that alien life exists?,evidence alien life space,1.0


<h3 style = "font-size:30px; font-family:Garamond ; font-weight : normal; background-color: #f6f5f5 ; color : #fe346e; text-align: center; border-radius: 100px 100px;">Apply Text preprocessing techniques on question1</h3>

In [20]:
%%time
dataH["question1"] = dataH.question1.apply(remove_punctuation)
dataH["question1"] = dataH.question1.apply(remove_numbers)
dataH["question1"] = dataH.question1.apply(remove_html_tags)
dataH["question1"] = dataH.question1.apply(remove_urls)
dataH["question1"] = dataH.question1.apply(convert_to_lowercase)
dataH["question1"] = dataH.question1.apply(apply_tokenization_and_remove_stopwords)
dataH["question1"] = dataH.question1.apply(apply_lemmatization)

CPU times: user 25.6 ms, sys: 678 µs, total: 26.2 ms
Wall time: 26 ms


In [21]:
class TextMatching:
    def __init__(self):
        self.question5DocRank = {}
        self.question2DocRank = {}
    
    def getTop5TextMatching(self, dataset):
        ids = dataset.id.to_list()
        questions = dataset.question1.to_list()
        for (doc, question1) in zip(ids, questions):
            docRank = {}
            for w in question1.split():
                if w in invertedFileMap:
                    for (d, s) in invertedFileMap[w]:
                        if d not in docRank:
                            docRank[d] = s
                        else:
                            docRank[d] += s
            top5Questions = sorted(docRank, key=docRank.get, reverse=True)[:5]
            self.question5DocRank[(doc, question1)] = top5Questions
        return self.question5DocRank
    
    def getTop2TextMatching(self, dataset):
        ids = dataset.id.to_list()
        questions = dataset.question1.to_list()
        for (doc, question1) in zip(ids, questions):
            docRank = {}
            for w in question1.split():
                if w in invertedFileMap:
                    for (d, s) in invertedFileMap[w]:
                        if d not in docRank:
                            docRank[d] = s
                        else:
                            docRank[d] += s
            top2Questions = sorted(docRank, key=docRank.get, reverse=True)[:2]
            self.question2DocRank[(doc, question1)] = top2Questions
        return self.question2DocRank

In [22]:
%%time
tmInstance = TextMatching()
top5Docs = tmInstance.getTop5TextMatching(dataH)

CPU times: user 357 ms, sys: 9.61 ms, total: 366 ms
Wall time: 365 ms


In [23]:
# Return top 5 questions instead of document ids
with open("invertedFileTextMatchingOutput.txt", "w") as f:
    for (doc, q) in top5Docs:
        q1 = originalData[originalData.id == doc].question1.to_list()[0]
        q2 = originalData[originalData.id.isin(top5Docs[(doc, q)])].question2.to_list()
        f.write("******************\n")
        f.write(f"{q1}\n")
        f.write(f"{q2}\n")

<h3 style = "font-size:60px; font-family:Garamond ; font-weight : normal; background-color: #f6f5f5 ; color : #fe346e; text-align: center; border-radius: 100px 100px;">Performance Evaluation</h3>
<br>
<br>
<span style="font-size:25px; font-family:Garamond ; font-weight : normal; background-color: #f6f5f5 ; color : #fe346e; border-radius: 30px 30px">Probability top5 vs top2</span>

In [24]:
class Performance:
    def __init__(self):
        self.totalLen = 0
        self.truth = 0
    
    def getProbability(self, dataset, questionDoc):
        ids = dataset.id.to_list()
        self.totalLen = len(ids)
        for (doc, q) in questionDoc:
            if doc in questionDoc[(doc,q)]:
                self.truth += 1
        return (self.truth / self.totalLen)

In [25]:
secondInstance = TextMatching()
top2Docs = secondInstance.getTop2TextMatching(dataH)

prob5 = Performance()
top5Prob = prob5.getProbability(dataH, top5Docs)
prob2 = Performance()
top2Prob = prob2.getProbability(dataH, top2Docs)

print(f"Probability for ground truth with top 5 docs is {top5Prob}")
print(f"Probability for ground truth with top 2 docs is {top2Prob}")

Probability for ground truth with top 5 docs is 0.37
Probability for ground truth with top 2 docs is 0.23


<h3 style = "font-size:60px; font-family:Garamond ; font-weight : normal; background-color: #f6f5f5 ; color : #fe346e; text-align: center; border-radius: 100px 100px;">Search question</h3>

In [26]:
def searchQuestionMatch(question):
    # it should go through text preprocessing phase
    question = remove_punctuation(question)
    question = remove_numbers(question)
    question = remove_html_tags(question)
    question = remove_urls(question)
    question = convert_to_lowercase(question)
    question = apply_tokenization_and_remove_stopwords(question)
    question = apply_lemmatization(question)
    
    wordList = question.split()
    docRank = {}
    for w in wordList:
        if w in invertedFileMap:
            for (d, s) in invertedFileMap[w]:
                if d not in docRank:
                    docRank[d] = s
                else:
                    docRank[d] += s
    top5 = sorted(docRank, key=docRank.get, reverse=True)[:5]
    return top5

In [27]:
question = input("Enter your question: ")
topMatches = searchQuestionMatch(question)

Enter your question: How to do MBBS?


In [28]:
# TODO: Return the questions instead of document id
tmatches = originalData[originalData.id.isin(topMatches)].question2.to_list()
tmatches

['What do i do after my MBBS ?',
 'What is the process of getting a surgical residency in UK after completing MBBS from India?',
 'Is it true that in order to get admission to usa after mbbs in india, ur medical college should be aiims only or u enter only on the basis of usmle? Basically does being from a good medical college in india guarantee you admission in usa or canada or some other country after mbbs or does it help?',
 'I am first year MBBS student(ST) in one of new Aiims.I feel inferior to others due to my rank in entrance exam.can I get good results in MBBS exams?',
 'How can I transfer from my MBBS medical college to another in Maharashtra after the 1st year of the MBBS?']

<h3 style = "font-size:60px; font-family:Garamond ; font-weight : normal; background-color: #f6f5f5 ; color : #fe346e; text-align: center; border-radius: 100px 100px;">Sentence Embedding</h3>
<br>
<h4 style = "font-size:40px; font-family:Garamond ; font-weight : normal; background-color: #f6f5f5 ; color : #fe346e; text-align: center; border-radius: 100px 100px;">Loading pretrained word embeddings and mapping them</h4>

In [ ]:
# uncomment below lines to download word embeddings from glove
# !wget http://nlp.stanford.edu/data/glove.6B.zip
# !unzip -q glove.6B.zip

In [29]:
glovePath = "glove.6B.300d.txt"

In [30]:
%%time
wordEmbeddings = {}
with open(glovePath) as f:
    for line in f:
        wordVec = line.split()
        word = wordVec[0]
        vec = list(map(float, wordVec[1:]))
        wordEmbeddings[word] = vec

CPU times: user 13.4 s, sys: 2.78 s, total: 16.2 s
Wall time: 18.8 s


<span style="color:brown; font-size:16px">We already applied text preprocessing on question2 which is our data and question1 in dataH which is our queries or test data.</span>
<br> 

<h4 style = "font-size:40px; font-family:Garamond ; font-weight : normal; background-color: #f6f5f5 ; color : #fe346e; text-align: center; border-radius: 100px 100px;">Generating average Sentence Embeddings</h4>

In [31]:
class GenerateAverageSentenceEmbedding:
    def __init__(self, dimensionLen):
        self.averageSentenceEmbedding = {}
        self.sumWordEmbeddingVector = np.zeros(dimensionLen)
    
    def generateEmbeddings(self, sentenceList, preTrainedWordEmbeddings):
        for sentence in sentenceList:
            lenWords = 0
            for w in sentence.split():
                lenWords += 1
                # sum of all the vectors for that word embedding
                if w in preTrainedWordEmbeddings:
                    self.sumWordEmbeddingVector = np.add(self.sumWordEmbeddingVector, preTrainedWordEmbeddings.get(w))
            self.averageSentenceEmbedding[sentence] = (self.sumWordEmbeddingVector / lenWords)
        return self.averageSentenceEmbedding

<h4 style = "font-size:40px; font-family:Garamond ; font-weight : normal; background-color: #f6f5f5 ; color : #fe346e; text-align: center; border-radius: 100px 100px;">Generate sentence Embedding for data from question2</h4>

In [32]:
%%time
q2List = data.question2.to_list()
q2Instance = GenerateAverageSentenceEmbedding(len(wordEmbeddings.get("of")))
dataEmbeddings = q2Instance.generateEmbeddings(q2List, wordEmbeddings)

CPU times: user 15.1 s, sys: 557 ms, total: 15.7 s
Wall time: 16.1 s


<h4 style = "font-size:40px; font-family:Garamond ; font-weight : normal; background-color: #f6f5f5 ; color : #fe346e; text-align: center; border-radius: 100px 100px;">Generate Sentence Embedding for input query data from question1 in dataH</h4>

In [33]:
%%time
q1List = dataH.question1.to_list()
queryInstance = GenerateAverageSentenceEmbedding(len(wordEmbeddings.get("of")))
queryEmbeddings = queryInstance.generateEmbeddings(q1List, wordEmbeddings)

CPU times: user 15.1 ms, sys: 5.21 ms, total: 20.3 ms
Wall time: 24.3 ms


<h4 style = "font-size:40px; font-family:Garamond ; font-weight : normal; background-color: #f6f5f5 ; color : #fe346e; text-align: center; border-radius: 100px 100px;">Calculate Cosine Similarity</h4>
<h5 style = "font-size:20px; font-family:Garamond ; font-weight : normal; background-color: #f6f5f5 ; color : #fe346e; text-align: center; border-radius: 10px 10px;">The cosine similarity gives the closeness of each input query to our data</h5>

In [34]:
def getCosineSimilarity(x, y):
    cosineSimilarity = np.dot(x, y) / ((sum(x**2)**0.5) * (sum(y**2)**0.5))
    return cosineSimilarity

<h4 style = "font-size:40px; font-family:Garamond ; font-weight : normal; background-color: #f6f5f5 ; color : #fe346e; text-align: center; border-radius: 100px 100px;">Text Matching</h4>

In [35]:
%%time
resultTop5SE = {}
resultTop2SE = {}
for k, q in queryEmbeddings.items():
    cose = []
    for dataEmb in dataEmbeddings.values():
        cose.append(getCosineSimilarity(q, dataEmb))  
    resultTop5SE[k] = [np.argpartition(cose, -5)[-5:]]
    resultTop2SE[k] = [np.argpartition(cose, -2)[-2:]]

CPU times: user 11min 54s, sys: 3.59 s, total: 11min 57s
Wall time: 11min 57s


In [36]:
with open("sentenceEmbeddingOutput.txt", "w") as f:
    for q, v in resultTop5SE.items():
        docId = dataH[dataH.question1 == q].id.to_list()[0]
        originalInputQuery = originalData[originalData.id==docId].question1.to_list()[0]
        q2 = originalData.iloc[resultTop5SE[q][0]].question2.to_list()
        f.write("******************\n")
        f.write(f"{originalInputQuery}\n")
        f.write(f"{q2}\n")

<h3 style = "font-size:60px; font-family:Garamond ; font-weight : normal; background-color: #f6f5f5 ; color : #fe346e; text-align: center; border-radius: 100px 100px;">Performance Evaluation</h3>
<br>
<br>
<span style="font-size:25px; font-family:Garamond ; font-weight : normal; background-color: #f6f5f5 ; color : #fe346e; border-radius: 30px 30px">Probability top5 vs top2</span>

In [37]:
class ProbabilitySentenceEmbedding:
    def __init__(self, totalLen):
        self.totalLen = totalLen
        self.truth = 0
    
    def getProbability(self, result, dataH, originalData):
        for q, f in result.items():
            docIds = dataH[dataH.question1 == q].id.to_list()[0]
            originalDocIds = originalData.iloc[result[q][0]].id.to_list()
            if docIds in originalDocIds:
                self.truth += 1
        return (self.truth / self.totalLen)

In [39]:
top5SEInstance = ProbabilitySentenceEmbedding(len(q1List))
top5SEProb = top5SEInstance.getProbability(resultTop5SE, dataH, originalData)
top2SEIntance = ProbabilitySentenceEmbedding(len(q1List))
top2SEProb = top2SEIntance.getProbability(resultTop2SE, dataH, originalData)
print(f"Sentence Embedding Probability for ground truth with top 5 docs is {top5SEProb}")
print(f"Sentence Embedding Probability for ground truth with top 2 docs is {top2SEProb}")

Sentence Embedding Probability for ground truth with top 5 docs is 0.28
Sentence Embedding Probability for ground truth with top 2 docs is 0.12


<h3 style = "font-size:60px; font-family:Garamond ; font-weight : normal; background-color: #f6f5f5 ; color : #fe346e; text-align: center; border-radius: 100px 100px;">Search question Sentence Embedding</h3>

In [40]:
def searchQuestionSentenceEmbedding(question, wordEmbeddings, dataEmbeddings, originalData):
    # it should go through text preprocessing phase
    question = remove_punctuation(question)
    question = remove_numbers(question)
    question = remove_html_tags(question)
    question = remove_urls(question)
    question = convert_to_lowercase(question)
    question = apply_tokenization_and_remove_stopwords(question)
    question = apply_lemmatization(question)
    
    instanceQ = GenerateAverageSentenceEmbedding(len(wordEmbeddings.get("of")))
    qEmbeddings = instanceQ.generateEmbeddings([question], wordEmbeddings)
    
    searchQuestionTop5 = {}
    for k, q in qEmbeddings.items():
        cose = []
        for dataEmb in dataEmbeddings.values():
            cose.append(getCosineSimilarity(q, dataEmb))  
        searchQuestionTop5[k] = [np.argpartition(cose, -5)[-5:]]
    
        t = originalData.iloc[searchQuestionTop5[k][0]].question2.to_list()
    return t

In [41]:
%%time
question = input("Enter your Question: ")
top5MatchesSentenceEmbedding = searchQuestionSentenceEmbedding(
    question, 
    wordEmbeddings.copy(), 
    dataEmbeddings,
    originalData
)

Enter your Question: How to do MBBS?
CPU times: user 7.15 s, sys: 255 ms, total: 7.41 s
Wall time: 17.7 s


In [42]:
top5MatchesSentenceEmbedding

["I don't beleive I am bulimic, but I force throw up atleast once a day after I eat something and feel guilty. Should I tell somebody, and if so who?",
 'How do you control your horniness?',
 'Would a second airport in Sydney, Australia be needed if a high-speed rail link was created between Melbourne and Sydney?',
 'Which level of prepration is enough for the exam jlpt5?',
 "What's the difference between an artist and an artisan?"]

<h3 style = "font-size:60px; font-family:Garamond ; font-weight : normal; background-color: #f6f5f5 ; color : #fe346e; text-align: center; border-radius: 100px 100px;">Sentence Embedding with DownWeight Frequent Words</h3>

In [43]:
class GenerateDownWeightSentenceEmbedding:
    def __init__(self, dimensionLen):
        self.downWeightSentenceEmbedding = {}
        self.sumWordEmbeddingVector = np.zeros(dimensionLen)
        self.probWMap = {}
    
    def probabilityW(self, sentenceList):
        countTotalWords = 0
        for sentence in sentenceList:
            for w in sentence.split():
                countTotalWords += 1
                if w not in self.probWMap:
                    self.probWMap[w] = 1
                else:
                    self.probWMap[w] += 1
        for w in self.probWMap:
            self.probWMap[w] = self.probWMap[w] / countTotalWords
        
            
    def generateEmbeddings(self, sentenceList, preTrainedWordEmbeddings):
        a = 10 ** -4
        # First calculate probability of w
        self.probabilityW(sentenceList)
        for sentence in sentenceList:
            lenWords = 0
            for w in sentence.split():
                lenWords += 1
                downWeight = (a/(a + self.probWMap[w]))
                if w in preTrainedWordEmbeddings:
                    self.sumWordEmbeddingVector = self.sumWordEmbeddingVector * downWeight
                    # sum of all the vectors for that word embedding
                    self.sumWordEmbeddingVector = np.add(self.sumWordEmbeddingVector,preTrainedWordEmbeddings.get(w))
            self.downWeightSentenceEmbedding[sentence] = self.sumWordEmbeddingVector / lenWords
        return self.downWeightSentenceEmbedding

<h4 style = "font-size:40px; font-family:Garamond ; font-weight : normal; background-color: #f6f5f5 ; color : #fe346e; text-align: center; border-radius: 100px 100px;">Generate sentence Embedding for data from question2</h4>

In [44]:
%%time
q2List = data.question2.to_list()
q2Instance = GenerateDownWeightSentenceEmbedding(len(wordEmbeddings.get("of")))
downWeightDataEmbeddings = q2Instance.generateEmbeddings(q2List, wordEmbeddings.copy())

CPU times: user 16.4 s, sys: 742 ms, total: 17.1 s
Wall time: 17.7 s


<h4 style = "font-size:40px; font-family:Garamond ; font-weight : normal; background-color: #f6f5f5 ; color : #fe346e; text-align: center; border-radius: 100px 100px;">Generate Sentence Embedding for input query data from question1 in dataH</h4>

In [45]:
%%time
q1List = dataH.question1.to_list()
queryInstance = GenerateDownWeightSentenceEmbedding(len(wordEmbeddings.get("of")))
queryEmbeddings = queryInstance.generateEmbeddings(q1List, wordEmbeddings)

CPU times: user 12.2 ms, sys: 9.16 ms, total: 21.3 ms
Wall time: 24.5 ms


<h4 style = "font-size:40px; font-family:Garamond ; font-weight : normal; background-color: #f6f5f5 ; color : #fe346e; text-align: center; border-radius: 100px 100px;">Calculate Cosine Similarity</h4>

In [46]:
%%time
resultTop5 = {}
resultTop2 = {}
for k, q in queryEmbeddings.items():
    cose = []
    for dataEmb in downWeightDataEmbeddings.values():
        cose.append(getCosineSimilarity(q, dataEmb))  
    resultTop5[k] = [np.argpartition(cose, -5)[-5:]]
    resultTop2[k] = [np.argpartition(cose, -2)[-2:]]

CPU times: user 11min 53s, sys: 3.36 s, total: 11min 56s
Wall time: 11min 55s


<h3 style = "font-size:60px; font-family:Garamond ; font-weight : normal; background-color: #f6f5f5 ; color : #fe346e; text-align: center; border-radius: 100px 100px;">Performance Evaluation</h3>
<br>
<br>
<span style="font-size:25px; font-family:Garamond ; font-weight : normal; background-color: #f6f5f5 ; color : #fe346e; border-radius: 30px 30px">Probability top5 vs top2</span>

In [47]:
class DownWeightSEProbability:
    def __init__(self, totalLen):
        self.totalLen = totalLen
        self.truth = 0
        
    def getProbability(self, result, dataH, originalData):
        
        for q, f in result.items():
            c = 0
            docIds = dataH[dataH.question1 == q].id.to_list()[0]
            originalDocIds = originalData.iloc[result[q][0]].id.to_list()
            if docIds in originalDocIds:
                self.truth += 1
        return (self.truth / self.totalLen)

In [48]:
dtop5Instance = DownWeightSEProbability(len(q1List))
dtop5 = dtop5Instance.getProbability(resultTop5, dataH, originalData)
dtop5

0.19

In [49]:
dtop2Instance = DownWeightSEProbability(len(q1List))
dtop2 = dtop2Instance.getProbability(resultTop2, dataH, originalData)
dtop2

0.04

In [50]:
print(f"The probability of top 5 is {dtop5}")
print(f"The probability of top 2 is {dtop2}")

The probability of top 5 is 0.19
The probability of top 2 is 0.04


<h3 style = "font-size:30px; font-family:Garamond ; font-weight : normal; background-color: #f6f5f5 ; color : #fe346e; text-align: center; border-radius: 100px 100px;">Matched questions</h3>


In [51]:
with open("downWeightOutput.txt", "w") as f:
    for q, v in resultTop5.items():
        docId = dataH[dataH.question1 == q].id.to_list()[0]
        originalInputQuery = originalData[originalData.id==docId].question1.to_list()[0]
        q2 = originalData.iloc[resultTop5[q][0]].question2.to_list()
        f.write("******************\n")
        f.write(f"{originalInputQuery}\n")
        f.write(f"{q2}\n")

<h3 style = "font-size:60px; font-family:Garamond ; font-weight : normal; background-color: #f6f5f5 ; color : #fe346e; text-align: center; border-radius: 100px 100px;">Search Question</h3>

In [52]:
def searchQuestionDownWeight(question, wordEmbeddings, downWeightDataEmbeddings, originalData):
    # it should go through text preprocessing phase
    question = remove_punctuation(question)
    question = remove_numbers(question)
    question = remove_html_tags(question)
    question = remove_urls(question)
    question = convert_to_lowercase(question)
    question = apply_tokenization_and_remove_stopwords(question)
    question = apply_lemmatization(question)
    
    instance = GenerateDownWeightSentenceEmbedding(len(wordEmbeddings.get("of")))
    qEmbeddings = instance.generateEmbeddings([question], wordEmbeddings)
    
    searchQuestionTop5 = {}
    for k, q in qEmbeddings.items():
        cose = []
        for dataEmb in downWeightDataEmbeddings.values():
            cose.append(getCosineSimilarity(q, dataEmb))  
        searchQuestionTop5[k] = [np.argpartition(cose, -5)[-5:]]
    
        t = originalData.iloc[searchQuestionTop5[k][0]].question2.to_list()
    return t

In [53]:
%%time
question = input("Enter Question: ")
downWeightTop5MatchedQuestions = searchQuestionDownWeight(
    question, 
    wordEmbeddings.copy(), 
    downWeightDataEmbeddings,
    originalData
)


Enter Question: How to do MBBS?
CPU times: user 7.12 s, sys: 325 ms, total: 7.44 s
Wall time: 13.6 s


In [54]:
downWeightTop5MatchedQuestions

['How can I escape boredom?',
 'How is Reliance Jio providing free unlimited 4G data when other companies charge high?',
 'How does one start a small business?',
 "What's the difference between an artist and an artisan?",
 "What is the meaning of Hindi word 'Gaddar'?"]